In [1]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()

magic = [team for team in nba_teams if team['abbreviation'] == 'ORL'][0]
magic_id = magic['id']
print(f'magic id: {magic_id}')

magic id: 1610612753


In [2]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=magic_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']

In [3]:
from nba_api.stats.endpoints import playbyplay
import time
import pandas as pd
dfs = []
for game in games:
    dfs.append(playbyplay.PlayByPlay(game['GAME_ID']).get_data_frames()[0])
    time.sleep(1)

In [4]:
# 1 - made shot
# 2 - missed shot
# 3 - free throw, missed free throw starts with MISS
# 4 - rebound, denoting offensive or defensive
# 5 - steals and turnovers
# 6 - foul perpretrator and fouled
# 7 - goaltending?
# 8 - substitution

# lets start with these

In [17]:
final_df = pd.concat(dfs, ignore_index=True)
final_df.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0022500451,2,12,0,1,7:43 PM,12:00,None,Start of 1st Period (7:43 PM EST),None,None,None
1,0022500451,4,10,0,1,7:43 PM,12:00,Jump Ball Barnes vs. Carter Jr.: Tip to Jones,None,None,None,None
2,0022500451,7,2,1,1,7:43 PM,11:43,None,None,MISS Bane 27' 3PT Jump Shot,None,None
3,0022500451,8,4,0,1,7:43 PM,11:38,Quickley REBOUND (Off:0 Def:1),None,None,None,None
4,0022500451,11,2,1,1,7:43 PM,11:22,MISS Ingram 26' 3PT Jump Shot,None,None,None,None


In [68]:
import re
from collections import defaultdict

p = re.compile(r"(\s{2}|\' )([\w+ ]*)")
event_msg_action_types = defaultdict(set)

for index, play in final_df.iterrows(): # gets semantic meaning for EVENTMSGA codes sourced from nba_api docs
    if play['EVENTMSGTYPE'] != 1:
        continue
    description = play['HOMEDESCRIPTION'] or play['VISITORDESCRIPTION']
    if description is None:
        continue
    match = p.search(description)
    if not match:
        continue 
    event_msg_action = match.group(2).rstrip().replace(' ', '_').upper()
    event_msg_action_types[play['EVENTMSGACTIONTYPE']].add(event_msg_action)

for code, action in sorted(event_msg_action_types.items()):
    print(f'\t{code} = {[item for item in action]}')

	1 = ['3PT_JUMP_SHOT', 'JUMP_SHOT']
	2 = ['3PT_RUNNING_JUMP_SHOT', 'RUNNING_JUMP_SHOT']
	3 = ['HOOK_SHOT']
	5 = ['LAYUP']
	6 = ['DRIVING_LAYUP']
	7 = ['DUNK']
	9 = ['DRIVING_DUNK']
	41 = ['RUNNING_LAYUP']
	43 = ['ALLEY_OOP_LAYUP']
	44 = ['REVERSE_LAYUP']
	47 = ['TURNAROUND_JUMP_SHOT']
	50 = ['RUNNING_DUNK']
	51 = ['REVERSE_DUNK']
	52 = ['ALLEY_OOP_DUNK']
	57 = ['DRIVING_HOOK_SHOT']
	58 = ['TURNAROUND_HOOK_SHOT']
	63 = ['FADEAWAY_JUMPER']
	66 = ['3PT_JUMP_BANK_SHOT', 'JUMP_BANK_SHOT']
	67 = ['HOOK_BANK_SHOT']
	71 = ['FINGER_ROLL_LAYUP']
	72 = ['PUTBACK_LAYUP']
	73 = ['DRIVING_REVERSE_LAYUP']
	74 = ['RUNNING_REVERSE_LAYUP']
	75 = ['DRIVING_FINGER_ROLL_LAYUP']
	76 = ['RUNNING_FINGER_ROLL_LAYUP']
	78 = ['FLOATING_JUMP_SHOT']
	79 = ['PULLUP_JUMP_SHOT', '3PT_PULLUP_JUMP_SHOT']
	80 = ['3PT_STEP_BACK_JUMP_SHOT', 'STEP_BACK_JUMP_SHOT']
	83 = ['FADEAWAY_BANK_SHOT']
	85 = ['TURNAROUND_BANK_SHOT']
	86 = ['TURNAROUND_FADEAWAY']
	87 = ['PUTBACK_DUNK']
	93 = ['DRIVING_BANK_HOOK_SHOT']
	96 = ['TURNARO

In [69]:
first = final_df[final_df['EVENTMSGTYPE'] == 1].drop_duplicates(subset='EVENTMSGACTIONTYPE', keep='first').sort_values(by="EVENTMSGACTIONTYPE")

In [70]:
final_df[(final_df['EVENTMSGACTIONTYPE'] == 1) & (final_df['EVENTMSGTYPE'] == 1)].head(50)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
9,0022500451,18,1,1,1,7:44 PM,10:34,None,None,Carter Jr. 3PT Jump Shot (3 PTS) (Bane 1 AST),3 - 2,-1
12,0022500451,22,1,1,1,7:45 PM,10:00,None,None,Bane 24' 3PT Jump Shot (3 PTS) (Banchero 1 AST),6 - 2,-4
71,0022500451,99,1,1,1,7:58 PM,5:02,Shead 27' 3PT Jump Shot (3 PTS) (Ingram 2 AST),None,None,14 - 11,-3
101,0022500451,143,1,1,1,8:04 PM,2:11,Dick 3PT Jump Shot (5 PTS) (Quickley 1 AST),None,None,20 - 16,-4
182,0022500451,263,1,1,2,8:21 PM,8:10,None,None,Richardson 3PT Jump Shot (3 PTS) (Black 1 AST),42 - 25,-17
185,0022500451,268,1,1,2,8:21 PM,7:43,None,None,Banchero 3PT Jump Shot (7 PTS) (Richardson 4 AST),45 - 25,-20
249,0022500451,357,1,1,2,8:39 PM,3:31,None,None,Banchero 25' 3PT Jump Shot (12 PTS) (Carter Jr...,56 - 43,-13
250,0022500451,359,1,1,2,8:40 PM,3:16,Murray-Boyles 24' 3PT Jump Shot (3 PTS) (Shead...,None,None,56 - 46,-10
257,0022500451,369,1,1,2,8:42 PM,2:21,None,None,Jones 3PT Jump Shot (3 PTS) (Bane 2 AST),60 - 48,-12
327,0022500451,463,1,1,3,9:12 PM,6:28,Shead 26' 3PT Jump Shot (15 PTS) (Mamukelashvi...,None,None,77 - 69,-8


In [71]:
final_df

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0022500451,2,12,0,1,7:43 PM,12:00,None,Start of 1st Period (7:43 PM EST),None,None,None
1,0022500451,4,10,0,1,7:43 PM,12:00,Jump Ball Barnes vs. Carter Jr.: Tip to Jones,None,None,None,None
2,0022500451,7,2,1,1,7:43 PM,11:43,None,None,MISS Bane 27' 3PT Jump Shot,None,None
3,0022500451,8,4,0,1,7:43 PM,11:38,Quickley REBOUND (Off:0 Def:1),None,None,None,None
4,0022500451,11,2,1,1,7:43 PM,11:22,MISS Ingram 26' 3PT Jump Shot,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
16330,0022500081,722,8,0,4,9:40 PM,0:06,None,None,SUB: Mitchell FOR Jaquez Jr.,None,None
16331,0022500081,723,8,0,4,9:40 PM,0:06,None,None,SUB: Jovic FOR Ware,None,None
16332,0022500081,726,2,79,4,9:41 PM,0:03,None,None,MISS Adebayo 28' 3PT Pullup Jump Shot,None,None
16333,0022500081,727,4,0,4,9:41 PM,0:03,Carter Jr. REBOUND (Off:5 Def:3),None,None,None,None
